In [1]:
import torch
import cv2
import numpy as np
from torchvision import transforms
from PIL import Image

In [2]:
# Charger le modèle pré-entraîné
model_path = "model_jit.pth"
model = torch.jit.load(model_path)
model.eval()

RecursiveScriptModule(
  original_name=FireDetectionModel
  (model): RecursiveScriptModule(
    original_name=ResNet
    (conv1): RecursiveScriptModule(original_name=Conv2d)
    (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
    (relu): RecursiveScriptModule(original_name=ReLU)
    (maxpool): RecursiveScriptModule(original_name=MaxPool2d)
    (layer1): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(
        original_name=Bottleneck
        (conv1): RecursiveScriptModule(original_name=Conv2d)
        (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
        (conv2): RecursiveScriptModule(original_name=Conv2d)
        (bn2): RecursiveScriptModule(original_name=BatchNorm2d)
        (conv3): RecursiveScriptModule(original_name=Conv2d)
        (bn3): RecursiveScriptModule(original_name=BatchNorm2d)
        (relu): RecursiveScriptModule(original_name=ReLU)
        (downsample): RecursiveScriptModule(
          original_name=Sequential
  

In [4]:
transform = transforms.Compose([
        transforms.RandomResizedCrop((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
])


In [5]:
def predict_fire(frame, model):

    frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))  # Conversion en RGB pour PIL

    # Appliquer les transformations
    input_tensor = transform(frame_pil).unsqueeze(0)

    # Envoyer le modèle et l'image sur GPU si disponible
    if torch.cuda.is_available():
        input_tensor = input_tensor.cuda()
        model = model.cuda()

    # Faire la prédiction
    with torch.no_grad():
        output = model(input_tensor)
        probabilities = torch.nn.functional.softmax(output, dim=1)
        predicted_class = torch.argmax(probabilities, 1)
        confidence = probabilities[0, predicted_class].item()

    classes = ["Fire", "No Fire", "Start Fire"]
    return classes[predicted_class.item()], confidence * 100


In [6]:
import cv2

video_path = "vids.mp4"
output_path = "output_vids.mp4"

# Initialiser la capture vidéo
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Erreur : Impossible de lire la vidéo.")
else:
    # Obtenir les propriétés de la vidéo
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Initialiser l'écriture de la vidéo
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Faire la prédiction sur la frame actuelle
        prediction, confidence = predict_fire(frame, model)

        # Ajouter le texte de la prédiction et le pourcentage
        text = f"{prediction}: {confidence:.2f}%"
        font = cv2.FONT_HERSHEY_SIMPLEX
        text_size = cv2.getTextSize(text, font, 1, 2)[0]
        text_x = (frame.shape[1] - text_size[0]) // 2  # Centrer horizontalement
        text_y = (frame.shape[0] + text_size[1]) // 2  # Centrer verticalement

        cv2.putText(frame, text, (text_x, text_y), font, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Ajouter la frame annotée à la vidéo de sortie
        out.write(frame)

    # Libérer les ressources
    cap.release()
    out.release()
    print(f"Vidéo annotée enregistrée sous {output_path}")



Vidéo annotée enregistrée sous output_vids.mp4
